In [4]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
from load_data import get_data, get_data_multi, get_crops, get_crops_multi,get_data_one_class
%matplotlib inline

In [ ]:
def time_normalization(x, time_dim):
    return (x-x.mean(dim=time_dim, keepdim=True))/(torch.sqrt(x.var(dim=time_dim, keepdim=True))+1e-8)
class FBCSP(nn.Module):
  def __init__(self):
    super(FBCSP, self).__init__()
    self.time_normalization = time_normalization
    self.conv_time = nn.Conv2d(1, 32, (75, 1))
    self.conv_spat = nn.Conv2d(32, 32, (1, 64), bias=False)
    self.batchnorm2 = nn.BatchNorm2d(32)
    self.pool = nn.AvgPool2d(kernel_size=(75, 1), stride=(25, 1))
    self.dropout = nn.Dropout2d(p=0.5)
    self.conv_class = nn.Conv2d(32, 2, (12, 1))
    self.score_ave = nn.AdaptiveAvgPool2d((1, 1))

  def forward(self, x):
    x = self.time_normalization(x, time_dim=2)
    h = self.conv_time(x)
    h = self.conv_spat(h)
    h = self.batchnorm2(h)
    h = self.pool(h)
    h = self.dropout(h)
    h = self.conv_class(h)
    h = self.score_ave(h)
    # print(h.size())
    # return h
    return h.squeeze()

import os, sys
os.chdir('/home/seigyo/Documents/pytorch/brain_decoder/FBCSPdata')
model2 = FBCSP()
# パラメータの読み込み
param = torch.load('publicBCImodel.pth')
model2.load_state_dict(param)

In [ ]:
X, y = get_data(id=99, event_code=[4,8,12], filter=[0.5, 45], t=[0., 4])
X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]).transpose(0,1,3,2)

In [ ]:
X_var = Variable(torch.from_numpy(X)).cuda()
model2.cuda()

In [ ]:
Y = model2(X_var)

In [ ]:
Y_np = Y.cpu().data.numpy()

In [ ]:
Y_predict = Y_np.argmax(axis=1)
print(Y_predict)
print(y)

In [ ]:
(Y_predict == y).sum()/len(y)

In [ ]:
acc_list_trainer = []
for i in range(1, 51):
    X, y = get_data(id=i, event_code=[4,8,12], filter=[0.5, 45], t=[0., 4])
    X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]).transpose(0,1,3,2)
    X_var = Variable(torch.from_numpy(X)).cuda()
    Y = model2(X_var)
    Y_np = Y.cpu().data.numpy()
    Y_predict = Y_np.argmax(axis=1)
    acc = (Y_predict == y).sum()/len(y)
    print(acc)
    acc_list_trainer.append(acc)
    

In [ ]:
acc_list_tester = []
for i in range(51, 100):
    X, y = get_data(id=i, event_code=[4,8,12], filter=[0.5, 45], t=[0., 4])
    X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]).transpose(0,1,3,2)
    X_var = Variable(torch.from_numpy(X)).cuda()
    Y = model2(X_var)
    Y_np = Y.cpu().data.numpy()
    Y_predict = Y_np.argmax(axis=1)
    acc = (Y_predict == y).sum()/len(y)
    print(acc)
    acc_list_tester.append(acc)
    

In [ ]:
print(sum(acc_list_trainer)/len(acc_list_trainer))
print(sum(acc_list_tester)/len(acc_list_tester))

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(acc_list_trainer, kde=False, rug=False, bins=10) 

In [ ]:
sns.distplot(acc_list_tester, kde=False, rug=False, bins=10)
plt.xlabel('accuracy', fontsize=18)
plt.ylabel('number of subject', fontsize=18)

In [ ]:
fine_model = model2.cuda()

for param in fine_model.parameters():
    param.requires_grad = False

In [ ]:
fine_model.conv_class = nn.Conv2d(32, 2, (12,1))
fine_model.cuda()